In [1]:
import subprocess
import re

In [2]:
# === Configuration ===
data_path = "/xustrg0/2024B8049"
run_number = "222753"
mask_path = "/UserData/fperakis/test_data_2025/utilities/empty_mask.npy"
output_path = "/UserData/fperakis/test_data_2025/processed"
poni_file = "/UserData/fperakis/test_data_2025/utilities/geometry_test.poni"
nbins = 150
n_phi = 72
n_chunks = 10  # total number of chunks

In [17]:
# === Submit array job ===
env_vars = ",".join([
    f"DATA_PATH={data_path}",
    f"RUN_NUMBER={run_number}",
    f"MASK_PATH={mask_path}",
    f"OUTPUT_PATH={output_path}",
    f"PONI_FILE={poni_file}",
    f"NBINS={nbins}",
    f"N_PHI={n_phi}",
    f"N_CHUNKS={n_chunks}"
])

submit_cmd = [
    "qsub", "-J", f"0-{n_chunks-1}",
    "-v", env_vars,
    "submit_chunk.pbs"
]

array_result = subprocess.run(submit_cmd, capture_output=True, text=True)

# === Extract array job ID ===
match = re.search(r"(\d+)", array_result.stdout)
if not match:
    raise RuntimeError("Failed to parse array job ID. Output:\n" + array_result.stdout)

array_job_id = match.group(1)
print(f"\nArray job submitted with ID: {array_job_id}")



Array job submitted with ID: 8418963


In [20]:
print(f"Submitting merge job after array job ID: {array_job_id}")

Submitting merge job after array job ID: 8418963


In [25]:
# === Submit dependent merge job ===
merge_result = subprocess.run(
    ["qsub", "-W", f"depend=afterok:{array_job_id}", "combine_chunks.pbs"],
    capture_output=True,
    text=True
)

print("\nSubmitting merge job...")
# print("STDOUT:")
# print(merge_result.stdout)
# print("STDERR:")
# print(merge_result.stderr)

if merge_result.returncode != 0:
    raise RuntimeError("Failed to submit merge job:\n" + merge_result.stderr)

print("Merge job submitted successfully.")


Submitting merge job...
Merge job submitted successfully.
